## 빅데이터분석 7주차과제 201910803 박채희

### 1-1 성적데이터로 DataFrame을 생성

In [39]:
import pyspark

In [40]:
myConf=pyspark.SparkConf()
spark=pyspark.sql.SparkSession.builder.master("local").appName("myApp").config(conf=myConf).getOrCreate()

In [42]:
marks=[
    "김하나, English, 100",
    "김하나, Math, 80",
    "임하나, English, 70",
    "임하나, Math, 100",
    "김갑돌, English, 82.3",
    "김갑돌, Math, 98.5"
]

In [43]:
rdd = spark.sparkContext.parallelize(marks).map(lambda x : x.split(','))

In [44]:
df = spark.createDataFrame(rdd, schema=['이름','과목','점수'])

In [45]:
df.show()

+------+--------+-----+
|  이름|    과목| 점수|
+------+--------+-----+
|김하나| English|  100|
|김하나|    Math|   80|
|임하나| English|   70|
|임하나|    Math|  100|
|김갑돌| English| 82.3|
|김갑돌|    Math| 98.5|
+------+--------+-----+



### 1-2 zscore 컬럼을 생성

In [56]:
import numpy as np
from pyspark.sql.functions import udf

X = df.select('점수').rdd.map(lambda x: float(x[0])).collect()
xbar=float(np.mean(X))
sx=float(np.std(X,ddof=1))

zscoreUdf = udf(lambda x: (float(x)-xbar)/sx)
   
df = df.withColumn('zscore',zscoreUdf(df['점수']))

In [57]:
df.show()

+------+--------+-----+-------------------+
|  이름|    과목| 점수|             zscore|
+------+--------+-----+-------------------+
|김하나| English|  100| 0.9020148231647499|
|김하나|    Math|   80| -0.662172731456202|
|임하나| English|   70| -1.444266508766678|
|임하나|    Math|  100| 0.9020148231647499|
|김갑돌| English| 82.3|-0.4822911626747928|
|김갑돌|    Math| 98.5| 0.7847007565681785|
+------+--------+-----+-------------------+



### 1-3 cdf 컬럼을 생성

In [82]:
from scipy.stats import norm

cdfUdf = udf(lambda x : float(norm.cdf(x)))
df = df.withColumn('cdf',cdfUdf(df['zscore']))

In [83]:
df.show()

+------+--------+-----+-------------------+-------------------+
|  이름|    과목| 점수|             zscore|                cdf|
+------+--------+-----+-------------------+-------------------+
|김하나| English|  100| 0.9020148231647499| 0.8164755032306112|
|김하나|    Math|   80| -0.662172731456202| 0.2539302639589459|
|임하나| English|   70| -1.444266508766678|0.07433201139097219|
|임하나|    Math|  100| 0.9020148231647499| 0.8164755032306112|
|김갑돌| English| 82.3|-0.4822911626747928|0.31479956212721416|
|김갑돌|    Math| 98.5| 0.7847007565681785| 0.7836854804484269|
+------+--------+-----+-------------------+-------------------+

